In [14]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display
import datetime
import threading
import logging
import time
import itertools
import smtplib
from email.mime.text import MIMEText
from collections import Counter
import csv
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import spotify_utils as spu
from operator import itemgetter
from os.path import join as pj

from sklearn.linear_model import LinearRegression,LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

from imblearn.over_sampling import RandomOverSampler


from math import sqrt

In [6]:
# constants
REFRESH_CACHE = False
seen_playlist_id = "5al4jEBoq01LPmFDuGDnq4"      # Automated: Reviewed Items
played_playlist_id = "7EHT9D4ygqDlyGfqcFvkUv"    # 5 Esh Played
inbox_playlist_id = "1xsuqA0HU4bSosdaPyVlWG"     # 1 Esh Review


In [7]:
if REFRESH_CACHE:
    sp = spu.spotify_connect()
    seen_tracks = spu.get_playlist_tracks(sp,seen_playlist_id,audio_features=True)
    played_tracks = spu.get_playlist_tracks(sp,played_playlist_id)
    review_tracks = spu.get_playlist_tracks(sp,inbox_playlist_id)
    seen_tracks.to_csv('seen_tracks.csv',index=False)
    played_tracks.to_csv('played_tracks.csv',index=False)
    played_tracks.to_csv('played_tracks.csv',index=False)
    review_tracks.to_csv('review_tracks.csv',index=False)

In [8]:
seen_tracks = pd.read_csv('seen_tracks.csv')
played_tracks = pd.read_csv('played_tracks.csv')["id"]
seen_tracks = seen_tracks.merge(played_tracks,how='left',on=["id"],indicator="played")
seen_tracks["played"] = seen_tracks["played"].apply(lambda i: 1 if i=="both" else 0)
seen_tracks = seen_tracks.dropna()
X = seen_tracks[spu.audio_features_to_use]
y = seen_tracks["played"]

In [9]:
y.value_counts()

0    8640
1     272
Name: played, dtype: int64

In [15]:
# Generic model evaluator
def evaluate_model(name,model,splitter,transformer,X,y):
    X,y = transformer(X,y)
    X_train, X_test, y_train, y_test = splitter(X,y,random_state=0)
    training_start = time.time()
    model.fit(X_train, y_train)
    training_end = time.time()
    training_duration = training_end - training_start
    training_score = "{:.2f}".format(model.score(X_train,y_train))
    test_score = "{:.2f}".format(model.score(X_test,y_test))
    y_test_pred = model.predict(X_test)
    tp = y_test_pred
    CM = confusion_matrix(y_test,y_test_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    return (name,training_duration,FP,FN,TP,TN)

def stratified_split(X,y,random_state):
    return train_test_split(X,y,stratify=y,test_size=0.25,random_state=random_state)

In [20]:
models = [("Dummy",DummyClassifier(random_state=0,strategy="constant",constant=0)),
          ("KNN",KNeighborsClassifier()),
          ("Logistic Regression",LogisticRegression()),
          ("Decision tree",DecisionTreeClassifier(random_state=0)),
          ("Random forests",RandomForestClassifier(random_state=0))
         ]
spliiters = [("RandSplit",train_test_split),
             ("StratSplit",stratified_split)
            ]
transformers = [("Identity",lambda X,y: (X,y)),
                ("Over-sample",RandomOverSampler(random_state=0).fit_resample)
               ]
results = []
for transformer_name,transformer in transformers:
    for splitter_name,splitter in spliiters:
        for model_name,model in models:
            results.append(evaluate_model(f'{model_name},{splitter_name},{transformer_name}',model,splitter,transformer,X,y))
print('\n'.join([','.join(map(str,res)) for res in results]))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

Dummy,RandSplit,Identity,0.0020017623901367188,0,73,0,2155
KNN,RandSplit,Identity,0.01600503921508789,0,73,0,2155
Logistic Regression,RandSplit,Identity,0.15770816802978516,0,73,0,2155
Decision tree,RandSplit,Identity,0.13010907173156738,89,71,2,2066
Random forests,RandSplit,Identity,2.6348989009857178,1,73,0,2154
Dummy,StratSplit,Identity,0.0010750293731689453,0,68,0,2160
KNN,StratSplit,Identity,0.01700282096862793,0,68,0,2160
Logistic Regression,StratSplit,Identity,0.21829676628112793,0,68,0,2160
Decision tree,StratSplit,Identity,0.12053179740905762,76,64,4,2084
Random forests,StratSplit,Identity,3.6953976154327393,2,68,0,2158
Dummy,RandSplit,Over-sample,0.0010006427764892578,0,2196,0,2124
KNN,RandSplit,Over-sample,0.03705453872680664,268,0,2196,1856
Logistic Regression,RandSplit,Over-sample,0.29563236236572266,960,545,1651,1164
Decision tree,RandSplit,Over-sample,0.1710829734802246,87,0,2196,2037
Random forests,RandSplit,Over-sample,4.5792076587677,3,0,2196,2121
Dummy,StratSplit,Ove

In [ ]:
# Evaluating different models


param_grid = [
    # {'classifier': [KNeighborsClassifier()], 'preprocessing': [StandardScaler(), None],
    #  'classifier__n_neighbors': [1,3,5]
    # },
    # {'classifier': [LinearRegression()], 'preprocessing': [StandardScaler(), None]
    # },
    # {'classifier': [DecisionTreeClassifier()], 'preprocessing': [StandardScaler(), None],
    #  'classifier__max_depth': [1,2,3,4]
    # },
    {'classifier': [RandomForestClassifier()],
     'preprocessing': [StandardScaler(),None],
     'classifier__max_features': [1, int(sqrt(len(features_to_use))), len(features_to_use)],
     'classifier__n_estimators': [1, 10, 100],
     'classifier__min_samples_split': [2, 5, 10]
    },
    {'classifier': [RandomForestClassifier()],
     'preprocessing': [None],
     'classifier__max_features': [len(features_to_use)],
     'classifier__n_estimators': [100],
     'classifier__min_samples_split': [10]
    }
    ]
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', RandomForestClassifier())], memory="cache_folder")
grid_search = GridSearchCV(pipe,param_grid,cv=5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)
grid_search.fit(X_train[features_to_use].astype(float),y_train)

print("Best params:\n{}\n".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Test-set score: {:.2f}".format(grid_search.score(X_test[features_to_use].astype(float), y_test)))

chosen_model = grid_search

In [ ]:
X_test_copy = X_test
X_test_copy["prediction"] = chosen_model.predict(X_test[features_to_use].astype(float))
X_test_copy["truth"] = y_test
test_errors = X_test_copy[~(X_test_copy["prediction"]==X_test_copy["truth"])]
display(test_errors[["artist_name","name","prediction","truth"]])

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri))
prediction_tracks = get_all_playlist_track_details(sp,prediction_playlist_id)
prediction_tracks_df = dict_list_to_dataframe(prediction_tracks)
prediction_tracks_df["prediction: is metal?"] = chosen_model.predict(prediction_tracks_df[features_to_use])
prediction_tracks_df["probability"] = chosen_model.predict_proba(prediction_tracks_df[features_to_use])[:,1]
display(prediction_tracks_df[["artist_name","name","prediction: is metal?","probability"]])

In [ ]:
def plot_feature_importances(data,model):
    n_features = data.shape[1]
    plt.barh(np.arange(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), features_to_use)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

plot_feature_importances(X,dtree)

In [ ]:
# Visualize the decision tree
from sklearn.tree import export_graphviz
graph_file = os.path.join(root_location,"tree.dot")
export_graphviz(dtree, out_file=graph_file, class_names=["Metallica", "Madonna"],
                feature_names=features_to_use, impurity=False, filled=True)
import graphviz
with open(graph_file) as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))